### Importing Libraries

In [231]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

### 1) Continuous Data Imputation

In [232]:
df_cont = pd.DataFrame({
    'Age': [25, 27, np.nan, 30, 29, np.nan],
    'Salary': [4000, np.nan, 5000, 5500, np.nan, 4500]
})
print("Original Data:")
df_cont

Original Data:


,Age,Salary
0,25.0,4000.0
1,27.0,NaN
2,NaN,5000.0
3,30.0,5500.0
4,29.0,NaN
5,NaN,4500.0


In [233]:
df_cont.isna().sum()

Age       2
Salary    2
dtype: int64

In [234]:
df_cont.value_counts()

Age   Salary
25.0  4000.0    1
30.0  5500.0    1
Name: count, dtype: int64

#### Univariate Imputation

(i) Mean Imputation

In [235]:
df = df_cont.copy()

imputer_mean = SimpleImputer(strategy='mean')
df['Age_mean'] = imputer_mean.fit_transform(df_cont[['Age']])
df['Salary_mean'] = imputer_mean.fit_transform(df_cont[['Salary']])
df1 = df
df1

,Age,Salary,Age_mean,Salary_mean
0,25.0,4000.0,25.00,4000.0
1,27.0,NaN,27.00,4750.0
2,NaN,5000.0,27.75,5000.0
3,30.0,5500.0,30.00,5500.0
4,29.0,NaN,29.00,4750.0
5,NaN,4500.0,27.75,4500.0


If you want to update existing data frame

In [236]:
df = df_cont.copy()

imputer_mean = SimpleImputer(strategy='mean')
df['Age'] = imputer_mean.fit_transform(df_cont[['Age']])
df['Salary'] = imputer_mean.fit_transform(df_cont[['Salary']])
df1 = df
df1

,Age,Salary
0,25.00,4000.0
1,27.00,4750.0
2,27.75,5000.0
3,30.00,5500.0
4,29.00,4750.0
5,27.75,4500.0


M1: Using Inplace

In [263]:
df = df_cont.copy()

# Find the issue here
df.fillna(df["Age"].mean(), inplace=True)
df

,Age,Salary
0,25.00,4000.00
1,27.00,27.75
2,27.75,5000.00
3,30.00,5500.00
4,29.00,27.75
5,27.75,4500.00


In [238]:
df = df_cont.copy()

df["Age"].fillna(df["Age"].mean(), inplace=True)
df["Salary"].fillna(df["Salary"].mean(), inplace=True)
df

,Age,Salary
0,25.00,4000.0
1,27.00,4750.0
2,27.75,5000.0
3,30.00,5500.0
4,29.00,4750.0
5,27.75,4500.0


reassigning to the same column (recommmended for smaller datasets)

In [264]:
df = df_cont.copy()

df["Age"] = df["Age"].fillna(df["Age"].median())
df["Salary"] = df["Salary"].fillna(df["Salary"].median())
df

,Age,Salary
0,25.0,4000.0
1,27.0,4750.0
2,28.0,5000.0
3,30.0,5500.0
4,29.0,4750.0
5,28.0,4500.0


(ii) Median Imputation

In [265]:
df = df1.copy()

imputer_median = SimpleImputer(strategy='median')

df['Age'] = imputer_median.fit_transform(df[['Age']])
df['Salary'] = imputer_median.fit_transform(df[['Salary']])
df

,Age,Salary
0,25.00,4000.0
1,27.00,4750.0
2,27.75,5000.0
3,30.00,5500.0
4,29.00,4750.0
5,27.75,4500.0


(iii) KNN imputation

In [266]:
df_cont

,Age,Salary
0,25.0,4000.0
1,27.0,NaN
2,NaN,5000.0
3,30.0,5500.0
4,29.0,NaN
5,NaN,4500.0


In [268]:
# Step 1: Scale
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_cont), columns=df_cont.columns)
df_scaled

,Age,Salary
0,-1.432078,-1.341641
1,-0.390567,NaN
2,NaN,0.447214
3,1.171700,1.341641
4,0.650945,NaN
5,NaN,-0.447214


In [270]:
# Step 1: Scale
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df_cont), columns=df_cont.columns)

# Step 2: KNN Imputation
imputer_knn = KNNImputer(n_neighbors=2)
df_knn_scaled = pd.DataFrame(imputer_knn.fit_transform(df_scaled),columns=df_cont.columns)

# Step 3: Inverse transform to original scale
df_knn = pd.DataFrame(scaler.inverse_transform(df_knn_scaled), columns=df_cont.columns)

print("KNN Imputed Data:")
df_knn

KNN Imputed Data:


,Age,Salary
0,25.0,4000.0
1,27.0,4750.0
2,27.5,5000.0
3,30.0,5500.0
4,29.0,4750.0
5,27.5,4500.0


(iv) MICE Imputation

In [271]:
imputer_iter = IterativeImputer(random_state=42)
df_mice = pd.DataFrame(imputer_iter.fit_transform(df_cont[['Age', 'Salary']]),
                       columns=['Age_MICE', 'Salary_MICE'])

print("MICE Imputed Data:")
df_mice

MICE Imputed Data:


,Age_MICE,Salary_MICE
0,25.000000,4000.000000
1,27.000000,4599.430334
2,28.337391,5000.000000
3,30.000000,5500.000000
4,29.000000,5199.412889
5,26.670263,4500.000000


### 2) Categorical Data Imputation

In [272]:

df_cat = pd.DataFrame({
    'Gender': ['Male', 'Female', np.nan, 'Male', np.nan],
    'Region': ['East', 'West', 'South', np.nan, 'West']
})
print("Original Data:\n")
df_cat

Original Data:



,Gender,Region
0,Male,East
1,Female,West
2,NaN,South
3,Male,NaN
4,NaN,West


Mode imputation

In [273]:

imputer_mode = SimpleImputer(strategy='most_frequent')
df_cat_mode = pd.DataFrame(imputer_mode.fit_transform(df_cat), columns=df_cat.columns)
print("After Mode Imputation:")
df_cat_mode

After Mode Imputation:


,Gender,Region
0,Male,East
1,Female,West
2,Male,South
3,Male,West
4,Male,West


Constant imputer

In [274]:
imputer_const = SimpleImputer(strategy='constant', fill_value='Unknown')
df_cat_const = pd.DataFrame(imputer_const.fit_transform(df_cat), columns=df_cat.columns)
df_cat_const

,Gender,Region
0,Male,East
1,Female,West
2,Unknown,South
3,Male,Unknown
4,Unknown,West


### 3) Mixed-Type Data Imputation

In [275]:

df_mixed = pd.DataFrame({
    'Age': [25, np.nan, 30, 28, np.nan],
    'Income': [4000, 4200, np.nan, 5000, 4800],
    'Gender': ['M', 'F', np.nan, 'M', 'F']
})
df_mixed

,Age,Income,Gender
0,25.0,4000.0,M
1,NaN,4200.0,F
2,30.0,NaN,NaN
3,28.0,5000.0,M
4,NaN,4800.0,F


In [276]:
# Replace Gender with numeric codes
df_mixed_enc = df_mixed.copy()
df_mixed_enc['Gender'] = df_mixed_enc['Gender'].map({'M': 1, 'F': 0}).astype(float)
df_mixed_enc

,Age,Income,Gender
0,25.0,4000.0,1.0
1,NaN,4200.0,0.0
2,30.0,NaN,NaN
3,28.0,5000.0,1.0
4,NaN,4800.0,0.0


In [277]:
imputer_mixed = IterativeImputer(random_state=42)

df_mixed_imp = pd.DataFrame(imputer_mixed.fit_transform(df_mixed_enc),
                            columns=df_mixed_enc.columns)
df_mixed_imp                           

,Age,Income,Gender
0,25.000000,4000.000000,1.0
1,25.601140,4200.000000,0.0
2,30.000000,5665.938632,0.5
3,28.000000,5000.000000,1.0
4,27.401663,4800.000000,0.0


In [278]:
# Set a threshold to determine the categorical variable
df_mixed_imp['Gender'] = np.where(df_mixed_imp['Gender'] > 0.5, 'M', 'F')
print("MICE Imputed Mixed Data:")
df_mixed_imp

MICE Imputed Mixed Data:


,Age,Income,Gender
0,25.000000,4000.000000,M
1,25.601140,4200.000000,F
2,30.000000,5665.938632,F
3,28.000000,5000.000000,M
4,27.401663,4800.000000,F


In [279]:
# Replace Gender with numeric codes
df_mixed_enc = df_mixed.copy()
df_mixed_enc['Gender'] = df_mixed_enc['Gender'].map({'M': 1, 'F': 0}).astype(float)
df_mixed_enc

# Fit the imputer
imputer_mixed = IterativeImputer(random_state=42)

df_mixed_imp = pd.DataFrame(imputer_mixed.fit_transform(df_mixed_enc),
                            columns=df_mixed_enc.columns)

# Set a threshold to determine the categorical variable
df_mixed_imp['Gender'] = np.where(df_mixed_imp['Gender'] > 0.5, 'M', 'F')
print("MICE Imputed Mixed Data:")
df_mixed_imp                         

MICE Imputed Mixed Data:


,Age,Income,Gender
0,25.000000,4000.000000,M
1,25.601140,4200.000000,F
2,30.000000,5665.938632,F
3,28.000000,5000.000000,M
4,27.401663,4800.000000,F


### 4) Time Series Data Imputation

In [281]:
dates = pd.date_range('2024-01-01', periods=7)
df_time = pd.DataFrame({
    'Date': dates,
    'Temp': [30, np.nan, 32, np.nan, 31, np.nan, 29]
})

print("Original Data:")
df_time

Original Data:


,Date,Temp
0,2024-01-01,30.0
1,2024-01-02,NaN
2,2024-01-03,32.0
3,2024-01-04,NaN
4,2024-01-05,31.0
5,2024-01-06,NaN
6,2024-01-07,29.0


In [252]:
df_time['Date'].dtypes

dtype('<M8[ns]')

In [253]:
df_time['Date'].loc[0]

Timestamp('2024-01-01 00:00:00')

Forward fill

In [282]:
df_time_ffill = df_time.ffill()

print("Forward Fill:")
df_time_ffill

Forward Fill:


,Date,Temp
0,2024-01-01,30.0
1,2024-01-02,30.0
2,2024-01-03,32.0
3,2024-01-04,32.0
4,2024-01-05,31.0
5,2024-01-06,31.0
6,2024-01-07,29.0


Backward fill

In [283]:
df_time_bfill = df_time.bfill()

print("\nBackward Fill:")
df_time_bfill


Backward Fill:


,Date,Temp
0,2024-01-01,30.0
1,2024-01-02,32.0
2,2024-01-03,32.0
3,2024-01-04,31.0
4,2024-01-05,31.0
5,2024-01-06,29.0
6,2024-01-07,29.0


Interpolation

In [284]:
df_time_interp = df_time.interpolate(method='linear')
print("Linear Interpolation:\n")
df_time_interp

Linear Interpolation:



,Date,Temp
0,2024-01-01,30.0
1,2024-01-02,31.0
2,2024-01-03,32.0
3,2024-01-04,31.5
4,2024-01-05,31.0
5,2024-01-06,30.0
6,2024-01-07,29.0


In [285]:
# Forward fill
df_time['Temp_ffill'] = df_time['Temp'].ffill()

# Backward fill
df_time['Temp_bfill'] = df_time['Temp'].bfill()

# Interpolation (linear by default)
df_time['Temp_interp'] = df_time['Temp'].interpolate()

print("Data with Imputed Columns:")
df_time

Data with Imputed Columns:


,Date,Temp,Temp_ffill,Temp_bfill,Temp_interp
0,2024-01-01,30.0,30.0,30.0,30.0
1,2024-01-02,NaN,30.0,32.0,31.0
2,2024-01-03,32.0,32.0,32.0,32.0
3,2024-01-04,NaN,32.0,31.0,31.5
4,2024-01-05,31.0,31.0,31.0,31.0
5,2024-01-06,NaN,31.0,29.0,30.0
6,2024-01-07,29.0,29.0,29.0,29.0


### Last Resort: Drop missing values

In [258]:
df = df_cont.copy()
df.dropna(inplace=True)
df

,Age,Salary
0,25.0,4000.0
3,30.0,5500.0
